In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import re, os


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
CSV_PATH = "/content/drive/MyDrive/yorumlar/isim_yorum_temiz_etiket.csv"

NON_COMPLAINT_CATEGORY = "Teşekkür"   # şikayet değilse zorunlu
FALLBACK_FOR_COMPLAINT = "Diğer"      # şikayet olup kural yoksa

FORCE_REBUILD_CATEGORY = True         # kategoriyi baştan ata
DEDUPLICATE_ROWS = True               # tekrarları temizle
DEDUP_SUBSET = ["isim", "yorum"]      # ya da sadece ["yorum"]

# Yardımcı (tahmin_*) sütunları dosyaya kaydetmeden önce sil
SAVE_HELPER_COLUMNS = False

# Ek dosya istemiyoruz:
SAVE_VERSIONED_COPY = False           # zaman damgalı kopya üretme
EXPORT_LOW_CONF = False               # düşük güvenli dosya üretme

# Düşük güven eşiği (sadece iç rapor için)
LOW_CONF_THRESHOLD = 1


In [ ]:
RULES = {
    "Su/Altyapı": [
        r"\bsu\s*(kesint|basınç|basinc|kok)\w*", r"\bkanalizasyon\w*", r"\bkanal\w*",
        r"\blogar\w*", r"\blağım\w*", r"\byağmur\s*suyu\w*", r"\byağmur\w*\s*(taşkın|baskın)\w*",
        r"\bızgara\w*", r"\bgider\w*", r"\bsızınt\w*", r"\bboru\w*", r"\bkaçak\w*"
    ],
    "Yol/Kaldırım": [
        r"\byol\w*", r"\byollar\w*", r"\bçukur\w*", r"\bçök\w*", r"\basfalt\w*", r"\byama\w*",
        r"\bparke\w*", r"\bkaldırım\w*", r"\bçamur\w*", r"\btoz\w*", r"\bkasis\w*", r"\brefüj\w*",
        r"\bçizgi\w*", r"\bsinyalizasyon\w*"
    ],
    "Aydınlatma": [
        r"\bışık\w*", r"\blamb\w*", r"\baydınlat\w*", r"\belektrik\s*direk\w*", r"\bdirek\w*\s*(yanm\w*|bozuk\w*|kır\w*)"
    ],
    "Çöp/Temizlik": [
        r"\bçöp\w*", r"\btemizlik\w*", r"\bkonteyner\w*", r"\bmoloz\w*", r"\batık\w*", r"\byığın\w*",
        r"\bçöp\s*topla\w*", r"\bgeri\s*dönüş\w*"
    ],
    "Ulaşım/Otopark": [
        r"\botopark\w*", r"\bpark\s*yeri\w*", r"\bpark\s*ihlal\w*", r"\bçift\s*sıra\w*", r"\btrafik\w*",
        r"\btıkan\w*", r"\bduran\s*araç\w*"
    ],
    "Toplu Taşıma": [
        r"\botobüs\w*", r"\bdurak\w*", r"\bsefer\w*", r"\bhat\W*\d+", r"\bminibüs\w*", r"\bdolmuş\w*",
        r"\bgecik\w*", r"\bsıklık\w*", r"\bçalışm\w*\s*hat\w*"
    ],
    "İnternet/Elektrik": [
        r"\binternet\w*", r"\bçekmiyor\w*", r"\bmodem\w*", r"\belektrik\w*", r"\bkesint\w*", r"\btrafo\w*"
    ],
    "İlaçlama/Haşere": [
        r"\bsivrisinek\w*", r"\bhaşere\w*", r"\bböcek\w*", r"\bilaçla\w*", r"\bkarasinek\w*", r"\blarva\w*", r"\byılan\w*", r"\bfare\w*"
    ],
    "Gürültü": [
        r"\bgürült\w*", r"\byüksek\s*ses\w*", r"\bpatpat\w*", r"\beksoz\w*", r"\bdüğün\s*salon\w*", r"\bgece\w*\s*rahats\w*"
    ],
    "Zabıta/Denetim": [
        r"\bzabıta\w*", r"\bdenetim\w*", r"\bseyyar\w*", r"\btabla\w*", r"\bruhsat\w*", r"\bişgal\w*", r"\bencümen\w*"
    ],
    "Park/Oyun Alanı": [
        r"\bpark\w*", r"\bçocuk\w*\s*oyun\w*", r"\bsalıncak\w*", r"\bkaydırak\w*", r"\bspor\w*\s*alan\w*",
        r"\byeşil\s*alan\w*", r"\bpeyzaj\w*", r"\bağaç\w*", r"\bbudama\w*", r"\bsulama\w*"
    ],
    "Hayvan/Sokak Hayvanları": [
        r"\bköpek\w*", r"\bkedi\w*", r"\bsokak\s*hayvan\w*", r"\bısır\w*", r"\bsaldır\w*", r"\bsürü\w*"
    ],
    "Okul/Çevresi": [
        r"\bokul\w*", r"\bokul\s*ön\w*", r"\bservis\w*", r"\büst\s*geçit\w*", r"\byaya\w*\s*geçit\w*"
    ],
    "İmar/İnşaat": [
        r"\bimar\w*", r"\binşa\w*", r"\bkaçak\w*", r"\byıkım\w*", r"\bisk[aâ]n\w*", r"\bruhsat\w*", r"\bhafriyat\w*"
    ],
    "Cami/İbadet Alanı": [
        r"\bcami\w*", r"\bcamii\w*", r"\bhoparlör\w*", r"\banons\w*", r"\bezan\w*"
    ],
    "Teşekkür": [
        r"\bteşekkür\w*", r"\belinize\s*sağlık\w*", r"\bemeğinize\s*sağlık\w*", r"\bsağol\w*", r"\bvar\s*ol\w*", r"\bgüzel\s*hizmet\w*"
    ],
}

CATEGORY_PRIORITY = [
    "Zabıta/Denetim","Su/Altyapı","Yol/Kaldırım","Aydınlatma","Çöp/Temizlik",
    "Ulaşım/Otopark","Toplu Taşıma","İlaçlama/Haşere","Gürültü",
    "Park/Oyun Alanı","Hayvan/Sokak Hayvanları","Okul/Çevresi",
    "İnternet/Elektrik","İmar/İnşaat","Cami/İbadet Alanı","Teşekkür",
]


In [ ]:
def _normalize_text(s: str) -> str:
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r"[^\wçğıöşü\s]", " ", s, flags=re.UNICODE)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def compile_rules(rules_dict):
    compiled = {}
    for cat, patterns in rules_dict.items():
        compiled[cat] = [re.compile(p, flags=re.IGNORECASE | re.UNICODE) for p in patterns]
    return compiled

COMPILED = compile_rules(RULES)

def apply_rules(text: str):
    t = _normalize_text(text)
    matched = {}
    for cat in CATEGORY_PRIORITY:
        hits = []
        for pat in COMPILED.get(cat, []):
            if pat.search(t):
                hits.append(pat.pattern)
        if hits: matched[cat] = hits
    if not matched: return None, {}, 0
    for cat in CATEGORY_PRIORITY:
        if cat in matched:
            return cat, matched, sum(len(v) for v in matched.values())
    return None, {}, 0


In [ ]:
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")

# Kolon isimleri
cols = {c.lower().strip(): c for c in df.columns}
def get_col(*cands):
    for c in cands:
        k = c.lower().strip()
        if k in cols: return cols[k]
    return None

col_isim   = get_col("isim")
col_yorum  = get_col("yorum")
col_temiz  = get_col("temiz yorum","temiz_yorum","temizyorum") or col_yorum
col_etiket = get_col("etiket")  # 1 = şikayet

# Tekrarları sil
if DEDUPLICATE_ROWS:
    subset = [c for c in DEDUP_SUBSET if c in df.columns]
    if subset:
        df = df.drop_duplicates(subset=subset, keep="first").reset_index(drop=True)

# Sütunları hazırla
if "kategori" not in df.columns or FORCE_REBUILD_CATEGORY:
    df["kategori"] = ""

# tahmin alanlarını sıfırla
for c in ["tahmin_kategori","tahmin_gerekce","tahmin_isabet_sayisi"]:
    if c in df.columns: del df[c]

def is_complaint(v) -> bool:
    if v is None: return False
    try:
        if pd.isna(v): return False
    except Exception:
        pass
    return str(v).strip() in {"1","1.0"} or v == 1 or v == 1.0

pred_cat, reasons, hits = [], [], []
for _, row in df.iterrows():
    text = row[col_temiz] if pd.notna(row[col_temiz]) else ""
    complaint = is_complaint(row[col_etiket]) if col_etiket else True

    if not complaint:
        best, matched, ucnt = NON_COMPLAINT_CATEGORY, {}, 0
    else:
        best, matched, ucnt = apply_rules(str(text))
        if not best: best, matched, ucnt = FALLBACK_FOR_COMPLAINT, {}, 0

    pred_cat.append(best)
    reasons.append(matched)
    hits.append(ucnt)

df["tahmin_kategori"] = pred_cat
df["tahmin_gerekce"] = [",".join([f"{k}:{len(v)}" for k,v in r.items()]) if r else "" for r in reasons]
df["tahmin_isabet_sayisi"] = hits

# Son kategori: tahmini yaz
df["kategori"] = df["tahmin_kategori"]


In [ ]:
if not SAVE_HELPER_COLUMNS:
    for c in ["tahmin_kategori","tahmin_gerekce","tahmin_isabet_sayisi"]:
        if c in df.columns: del df[c]

df.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")

print("Güncellendi ✅:", CSV_PATH)
print("Toplam satır:", len(df))
print("Teşekkür (şikayet değil):", (df["kategori"] == NON_COMPLAINT_CATEGORY).sum())
print("Şikayet (kurallı/Diğer):", (df["kategori"] != NON_COMPLAINT_CATEGORY).sum())


Güncellendi ✅: /content/drive/MyDrive/yorumlar/isim_yorum_temiz_etiket.csv
Toplam satır: 514
Teşekkür (şikayet değil): 213
Şikayet (kurallı/Diğer): 301


In [ ]:
base_dir = os.path.dirname(CSV_PATH)
base_name = os.path.basename(CSV_PATH)
name_no_ext, ext = os.path.splitext(base_name)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
versioned_out = os.path.join(base_dir, f"{name_no_ext}_KATEGORILI_{timestamp}{ext}")
lowconf_out   = os.path.join(base_dir, f"{name_no_ext}_DUSUK_GUVEN_{timestamp}{ext}")

# Zaman damgalı çıktı
df.to_csv(versioned_out, index=False, encoding="utf-8-sig")

# Ana dosyanın üstüne de yaz (opsiyonel)
if OVERWRITE_EXISTING:
    df.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")

# Düşük güvenli kayıtları ayrı kaydet
if len(low_conf_df) > 0:
    low_conf_df.to_csv(lowconf_out, index=False, encoding="utf-8-sig")

# Özet
print("Toplam satır:", len(df))
print("Şikayet değil (Teşekkür) sayısı:", (df["kategori"] == NON_COMPLAINT_CATEGORY).sum())
print("Şikayet (kurallı) sayısı:", (df["kategori"] != NON_COMPLAINT_CATEGORY).sum())
print("Zaman damgalı çıktı:", versioned_out)
if OVERWRITE_EXISTING:
    print("Ana CSV güncellendi:", CSV_PATH)
print("Düşük güvenli dosya:", lowconf_out if len(low_conf_df) else "—")


Toplam satır: 514
Şikayet değil (Teşekkür) sayısı: 213
Şikayet (kurallı) sayısı: 301
Zaman damgalı çıktı: /content/drive/MyDrive/yorumlar/isim_yorum_temiz_etiket_KATEGORILI_20250812_064640.csv
Ana CSV güncellendi: /content/drive/MyDrive/yorumlar/isim_yorum_temiz_etiket.csv
Düşük güvenli dosya: /content/drive/MyDrive/yorumlar/isim_yorum_temiz_etiket_DUSUK_GUVEN_20250812_064640.csv
